# CTMAP Tutorial

This notebook provides a complete walkthrough of CTMAP on the MERFISH dataset, including full training logs with progress bar and loss values.


## 1. Setup and Imports

This cell:
- Adds the project root directory to Python path (necessary for importing local modules)
- Sets environment variables
- Suppresses common warnings
- Imports required libraries and CTMAP modules

In [3]:
import sys
import os
# Environment settings
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Some cells have zero counts")
warnings.filterwarnings("ignore", category=FutureWarning)

# Core libraries
import torch
import numpy as np
import scanpy as sc
from sklearn.metrics import accuracy_score, normalized_mutual_info_score, adjusted_rand_score

# CTMAP modules
from CTMAP.model import CTMAP
from CTMAP.dataprocess import cell_type_encoder, anndata_preprocess, generate_dataloaders

print("\nAll imports successful!")


All imports successful!


## 2. Set random seed and device

This cell:
- Sets a fixed random seed for reproducibility
- Detects and selects GPU (if available) or CPU

In [4]:
seed = 0
print(f"\n{'='*70}")
print(f"CTMAP - MERFISH | Single Run with Seed {seed}")
print(f"{'='*70}")

torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


CTMAP - MERFISH | Single Run with Seed 0
Using device: cuda:0


## 3. Load data

This cell loads the example MERFISH dataset:
- scRNA-seq reference (`adata_rna.h5ad`)
- Spatial transcriptomics data (`adata_merfish.h5ad`)

Both files are included in the repository under `/dataset/MERFISH/`.

In [5]:
rna_path = "dataset/MERFISH/adata_rna.h5ad"
spatial_path = "dataset/MERFISH/adata_merfish.h5ad"

adata_r = sc.read(rna_path)
adata_s = sc.read(spatial_path)

print(f"Loaded scRNA-seq data: {adata_r.n_obs} cells, {adata_r.n_vars} genes")
print(f"Loaded spatial data: {adata_s.n_obs} spots, {adata_s.n_vars} genes")

Loaded scRNA-seq data: 30370 cells, 21043 genes
Loaded spatial data: 64373 spots, 160 genes


## 4. Preprocessing

This cell performs:
- Gene intersection between RNA and spatial data
- Adding source labels
- Cell type encoding
- Full preprocessing (normalization, log1p, per-batch MaxAbs scaling)
- Creating PyTorch DataLoaders

In [6]:
# Gene intersection
common_genes = adata_r.var_names.intersection(adata_s.var_names)
adata_rna = adata_r[:, common_genes].copy()
adata_spa = adata_s[:, common_genes].copy()

adata_rna.obs['source'] = 'RNA'
adata_spa.obs['source'] = 'MERFISH'
adata_rna.X = adata_rna.X.astype(np.float32)
adata_spa.X = adata_spa.X.astype(np.float32)

# Cell type encoding
_, _, cell_types = cell_type_encoder(adata_rna, adata_spa)

# Full preprocessing
adata_spa, adata_rna = anndata_preprocess(adata_spa, adata_rna)

# Create DataLoaders
rna_train_loader, st_train_loader, rna_test_loader, st_test_loader = generate_dataloaders(adata_spa, adata_rna)

print("\nPreprocessing completed.")
print(f"Final RNA data: {adata_rna.n_obs} cells, {adata_rna.n_vars} genes")
print(f"Final spatial data: {adata_spa.n_obs} spots, {adata_spa.n_vars} genes")

Unique RNA Labels: [ 0  1  2  3  4  5  6  7  8  9 10 11]
Unique SeqFISH Labels: [ 0  1  2  3  5  7 10 11 12]
common Labels: [ 0  1  2  3  5  7 10 11]
Number of classes: 13

Preprocessing completed.
Final RNA data: 30370 cells, 153 genes
Final spatial data: 64373 spots, 153 genes


## 5. Initialize model

This cell creates the CTMAP model instance with the appropriate input dimensions and hyperparameters.

In [7]:
r_dim = len(adata_rna)
s_dim = len(adata_spa)
rna_dim = adata_rna.shape[1]
st_dim = adata_spa.shape[1]

model = CTMAP(
    rna_dim=rna_dim,
    st_dim=st_dim,
    latent_dim=64,
    hidden_dim=256,
    mha_heads_1=4,
    mha_dim_1=256,
    mha_dim_2=128,
    mha_heads_2=4,
    class_num=len(np.unique(adata_rna.obs['cell_type'])),
    device=device
)

print("CTMAP model initialized.")

CTMAP model initialized.


## 6. Train the model

This cell runs the full training process.

**You will see:**
- Progress bar (tqdm)
- Loss values every 100 iterations
- Stage transition messages
- Final completion message

Training takes approximately 2-3 minutes on a GPU.

In [9]:
print("Starting training...\n")

truth_label, pred_label, truth_rna, rna_embeddings, st_embeddings = model.train(
    rna_dim=r_dim,
    spa_dim=s_dim,
    rna_train_loader=rna_train_loader,
    st_train_loader=st_train_loader,
    spatial_coor=adata_spa.obs[["X", "Y"]],
    rna_test_loader=rna_test_loader,
    st_test_loader=st_test_loader,
    lr=5e-4,
    maxiter=4000,
    miditer1=3000,
    log_interval=100,
    stage1_recon_weight=3.0,
    stage1_cls_weight=0.01,
    stage2_recon_weight=4.0,
    stage2_domain_weight=0.1,
    stage2_cls_weight=0.01
)

print("\nTraining completed!")

Starting training...

=== Starting Stage 1: Pretraining (Reconstruction + Classification) ===


Training:   3%|█▌                                                          | 105/4000 [00:05<02:41, 24.18it/s]

#Iter 100: recon_loss: 0.359234, cls loss: 2.482532, domain loss: 0.000000


Training:   5%|███                                                         | 202/4000 [00:09<02:14, 28.14it/s]

#Iter 200: recon_loss: 0.336991, cls loss: 2.486644, domain loss: 0.000000


...

#Iter 3900: recon_loss: 0.057458, cls loss: 2.485142, domain loss: 1.385459


Training: 100%|██████████████████████████████████████████████████████████▉| 3999/4000 [02:33<00:00, 44.62it/s]

#Iter 4000: recon_loss: 0.055800, cls loss: 2.484606, domain loss: 1.385740


Training: 100%|███████████████████████████████████████████████████████████| 4000/4000 [02:40<00:00, 24.91it/s]


Training completed!


## 7. Evaluation

This cell computes Accuracy, NMI, and ARI on the predicted spatial cell types.

In [10]:
truth_ct = [cell_types[i] for i in truth_label]
pred_ct = [cell_types[i] for i in pred_label]

accuracy = accuracy_score(truth_ct, pred_ct)
nmi = normalized_mutual_info_score(truth_ct, pred_ct)
ari = adjusted_rand_score(truth_ct, pred_ct)

print(f"\n{'='*70}")
print("FINAL RESULTS")
print(f"Accuracy: {accuracy:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}")
print(f"{'='*70}")


FINAL RESULTS
Accuracy: 0.8907, NMI: 0.7833, ARI: 0.7888
